# NRM Configuration/Manifest guide

This notebook documents NRM's configuration and manifest format. The two next cells are for setup purposes.

In [1]:
%%capture
cd ..

In [2]:
import json
import pprint

pp = pprint.PrettyPrinter(indent=4)

## Daemon configuration

`nrmd`'s configuration can be defined in the json/yaml/[Dhall](https://dhall-lang.org/) formats. Admissible values are defined in file [hsnrm/hsnrm/dhall/types/nrmd.dhall](hsnrm/hsnrm/dhall/types/nrmd.dhall).

In [19]:
%%bash
cat ./hsnrm/hsnrm/dhall/types/nrmd.dhall

-- ******************************************************************************
--  Copyright 2019 UChicago Argonne, LLC.
--  (c.f. AUTHORS, LICENSE)
--
--  SPDX-License-Identifier: BSD-3-Clause
-- ******************************************************************************
--
-- types used by nrmd's configuration.
--
--
let types =
    -- This import defines a few base types that are common to manifest and
    -- configuration formats.
      ./types.dhall

let Verbosity =
    -- Daemon verbosity:
    --   Error: Only report errors
    --   Info: Be verbose
    --   Debug: Report all that can possibly be reported
      < Error | Info | Debug >

let SensorBehavior =
    -- Sensor Behavior.
    -- IntervalBased: a sensor that measures an average value for the time
    -- period since its last reading.
    -- Cumulative: a sensor that measures a counter. NRM must substract between
    -- subsequent counter values to obtain a reading for the corresponding time
    -- period.
    -- Cum

Optional values are filled using defaults that can be found either in Dhall or JSON format: 

-[hsnrm/hsnrm/dhall/defaults/nrmd.dhall](hsnrm/hsnrm/dhall/defaults/nrmd.dhall) 
-[resources/defaults/nrmd.json](resources/defaults/nrmd.json)

In [21]:
%%bash
cat ./hsnrm/hsnrm/dhall/defaults/nrmd.dhall

let t = ../types/nrmd.dhall

in    { verbose = t.Verbosity.Error
      , logfile = "/tmp/nrm.log"
      , perfPath = "perf"
      , perfwrapperPath = "nrm-perfwrapper"
      , libnrmPath = None Text
      , downstreamCfg.downstreamBindAddress = "ipc:///tmp/nrm-downstream-event"
      , upstreamCfg =
          { upstreamBindAddress = "*", pubPort = +2345, rpcPort = +3456 }
      , raplCfg = Some
          { raplPath = "/sys/devices/virtual/powercap/intel-rapl"
          , raplActions = [ { microwatts = 1.0e8 }, { microwatts = 2.0e8 } ]
          , referencePower.microwatts = 2.5e8
          }
      , controlCfg = t.ControlCfg.ControlOff
      , passiveSensorFrequency.hertz = 1.0
      , extraStaticPassiveSensors = [] : List t.SensorKV
      , extraStaticActuators = [] : List t.ActuatorKV
      }
    : t.Cfg


In [31]:
%%bash
cat ./resources/defaults/nrmd.json | yq '' --yaml-output

extraStaticPassiveSensors: []
perfPath: perf
verbose: Error
logfile: /tmp/nrm.log
controlCfg: ControlOff
upstreamCfg:
  upstreamBindAddress: '*'
  rpcPort: 3456
  pubPort: 2345
libnrmPath: null
downstreamCfg:
  downstreamBindAddress: ipc:///tmp/nrm-downstream-event
perfwrapperPath: nrm-perfwrapper
extraStaticActuators: []
raplCfg:
  referencePower:
    microwatts: 250000000
  raplActions:
    - microwatts: 100000000
    - microwatts: 200000000
  raplPath: /sys/devices/virtual/powercap/intel-rapl
passiveSensorFrequency:
  hertz: 1


Attribute merging is always performed with those defaults. Example configurations are located in the `examples` folder:

In [27]:
%%bash
cat ./examples/nrmd/control.yaml

passiveSensorFrequency:
  hertz: 1
controlCfg:
  hint: Full
  learnCfg:
    horizon: 4000
  minimumWaitInterval:
    microseconds: 100000
  minimumControlInterval:
    microseconds: 100000
  referenceMeasurementRoundInterval: 6
  speedThreshold: 1.1
  staticPower:
    microwatts: 200000000
raplCfg:
  raplActions:
    - microwatts: 100000000
    - microwatts: 200000000
  raplPath: /sys/devices/virtual/powercap/intel-rapl
  referencePower:
    microwatts: 250000000


## Manifest configuration

Manifest files can also be defined either using Dhall, YAML, or JSON, and are at similar locations. 

In [28]:
%%bash
cat ./hsnrm/hsnrm/dhall/types/manifest.dhall

-- ******************************************************************************
--  Copyright 2019 UChicago Argonne, LLC.
--  (c.f. AUTHORS, LICENSE)
--
--  SPDX-License-Identifier: BSD-3-Clause
-- ******************************************************************************
let types =
    -- This import defines a few base types that are common to manifest and
    -- configuration formats.
      ./types.dhall

let Perfwrapper =
    -- Configuration for linux perf performance measurements. A fixed frequency
    -- is provided, along with a tentative upper bound on the maximum value of
    -- the sensor. This upper bound should be set to a low positive value if
    -- no a-priori information is known; NRM will then use a recursive-doubling
    -- strategy to maintain its own bound.
      { perfFreq : types.Frequency, perfLimit : Integer }

let Instrumentation =
    -- Libnrm instrumentation. The only attribute that configures this feature
    -- is a message rate limitation.
      { 

In [42]:
%%bash
cat ./hsnrm/hsnrm/dhall/defaults/manifest.dhall

-- ******************************************************************************
--  Copyright 2019 UChicago Argonne, LLC.
--  (c.f. AUTHORS, LICENSE)
--
--  SPDX-License-Identifier: BSD-3-Clause
-- ******************************************************************************

let t = ../types/manifest.dhall

in  { name = "default"
    , app =
        { perfwrapper = None t.Perfwrapper
        , instrumentation = None t.Instrumentation
        }
    }


Under-specified manifests like the one in our `workloads` above (with missing optional fields from the schema) fill missing values with defaults, which are located in file [resources/defaults/Manifest.json](../../resources/examples/default.json):

In [43]:
%%bash
cat ./hsnrm/hsnrm/dhall/defaults/manifest.dhall

-- ******************************************************************************
--  Copyright 2019 UChicago Argonne, LLC.
--  (c.f. AUTHORS, LICENSE)
--
--  SPDX-License-Identifier: BSD-3-Clause
-- ******************************************************************************

let t = ../types/manifest.dhall

in  { name = "default"
    , app =
        { perfwrapper = None t.Perfwrapper
        , instrumentation = None t.Instrumentation
        }
    }


The `dhall` and `dhall-to-json` utilities are available as convenience in this environment should you need them. Dhall is useful as a configuration language in itself:

In [44]:
%%bash
cat ./resources/defaults/manifest.json | yq '' --yaml-output

app:
  instrumentation: null
  perfwrapper: null
name: default


In [45]:
%%bash
cat ./examples/manifests/perfwrap.yaml

name: linuxperf-wrapped
app:
  perfwrapper:
    perfFreq:
      hertz: 1
    perfLimit: 100000
